In [8]:
import json
import os
from pathlib import Path
import re
import subprocess
import time

import jsonlines as jsonl
import numpy as np
import rich
import h5py



In [19]:
"""
Resume the training for all jobs under a certain number of epochs
"""
assert False

DATA_DIR = Path("./log_results/")
ORACLE = DATA_DIR / "oracle"
BASIC = DATA_DIR / "basic"

LIMIT = 60
checkpoints = list(ORACLE.glob("**/*.ckpt")) + list(BASIC.glob("**/*.ckpt"))

def epoch_no_from_ckpt_path(path):
    return int(re.search("epoch=(\w+)", path.name).group(1))

incomplete = [x for x in checkpoints if epoch_no_from_ckpt_path(x) < LIMIT]
incomplete_specific_info = [
    next(iter(x.parent.parent.parent.parent.parent.glob("**/specific_config.json"))) 
    for x in incomplete if "archive" not in str(x.parent.parent.parent.parent.parent)]
print(incomplete_specific_info)
if not incomplete_specific_info:
    print("No incomplete checkpoints found, nothing to resume.")

processes = []
for path in incomplete_specific_info:
    command = [
        "python", "./bin_main_launcher.py", "resume", str(Path.cwd() / path), "--gpu=a100", "--max_epochs=60"
    ]
    rich.print(command)
    processes.append(subprocess.Popen(command))
    
[p.wait() for p in processes]

[]
